# CNN
## Convolutional Neural Network　畳込みニューラルネットワーク

### (1) 入力層
28x28=784次元のベクトルが入力ベクトルです。<br>
数字の画像を784ピクセルに分割し、1次元=1ピクセルで情報を代入しています。

### (2) 畳込み層1 -> プーリング層1
2層準備。ゼロパディング(zero padding)という補完機能を利用しています。<br>
ファイルの畳込みに対して、周縁データが欠落するのですが、その補完のためにパディングを用います。<br>
（パディングの話で推測できるように、畳込み層では、入力の次元圧縮を行いません）<br>
プーリング(pooling)では、畳込みで返された数値を圧縮します。<br>
ここでは、最大プーリング(max pooling)という、最も基本的な手法を用いています。<br>
プーリングでも、パディングを行うことがあります。<br>
（重ね合わせがずれる領域が存在することがあるため）

* ゼロパディング(zero padding)とは、画像や帳票などで数値を表現する際、<br>
書式で指定した桁数に満たない部分をゼロで埋めることである。<br>
例えば「123」を5桁で表す場合、2桁足りない部分をゼロで埋めて「00123」と表記する<br>
* 最大プーリング(max pooling)とは、設定範囲のピクセルの中で最大値を選択する

### (3) 畳込み層2 -> プーリング層2
(2)の繰り返しでさらに圧縮 -> この段階で7x7=49次元まで落とす予定です。

### (4) 全結合層(高密度結合層)
抽出したプーリング層からの出力層を入力層に送り、NNのような処理をかけます。<br>
活性化関数はReLUです。

### (5) クラス分類処理
ソフトマックス関数の計算 -> 交差エントロピー誤差関数による評価

https://qiita.com/icoxfog417/items/5fd55fad152231d706c2



In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Network Parameters
n_input = 784
n_classes = 10
train_dropout = 0.8
test_dropout = 1.0

# Training Parameters
learning_rate = 0.001
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print(total_batch)
training_epochs = 30

# tf Graph input
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

550


In [3]:
# Create model


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def CNN(x, weights, biases, dropout):
    
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [4]:
# Construct model
y_pred = CNN(X, weights, biases, keep_prob)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1:
# loss = tf.reduce_mean(
#     -tf.reduce_sum(Y * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_pred))
# type 3:
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate model
pred = tf.argmax(y_pred, 1)
true = tf.argmax(Y, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
all_x_test = mnist.test.images
all_y_test = mnist.test.labels
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                X: batch_x,
                Y: batch_y,
                keep_prob: train_dropout
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = sess.run(accuracy,feed_dict={
            X: batch_x,
            Y: batch_y,
            keep_prob: train_dropout
        })
        test_acc = sess.run(accuracy,feed_dict={
            X: all_x_test,
            Y: all_y_test,
            keep_prob: test_dropout
        })
        print("Epoch:%2d Batch:%4d" % (epoch_i + 1, batch_i + 1),
              "train_acc=%.3f" % train_acc, "test_acc=%.3f" % test_acc,
              "train_cost=%5.3f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
acc = sess.run(
    accuracy,
    feed_dict={
        X: all_x_test,
        Y: all_y_test,
        keep_prob: test_dropout
    })
print("test accuracy=%.3f" % acc)
sess.close()

Epoch: 1 Batch: 550 train_acc=0.900 test_acc=0.950 train_cost=5309.987
Epoch: 2 Batch: 550 train_acc=0.980 test_acc=0.968 train_cost=716.799
Epoch: 3 Batch: 550 train_acc=1.000 test_acc=0.971 train_cost=361.856
Epoch: 4 Batch: 550 train_acc=0.940 test_acc=0.976 train_cost=234.321
Epoch: 5 Batch: 550 train_acc=0.970 test_acc=0.975 train_cost=151.020
Epoch: 6 Batch: 550 train_acc=0.980 test_acc=0.980 train_cost=110.864
Epoch: 7 Batch: 550 train_acc=0.980 test_acc=0.977 train_cost=82.911
Epoch: 8 Batch: 550 train_acc=0.990 test_acc=0.981 train_cost=65.452
Epoch: 9 Batch: 550 train_acc=1.000 test_acc=0.982 train_cost=53.947
Epoch:10 Batch: 550 train_acc=1.000 test_acc=0.981 train_cost=42.150
Epoch:11 Batch: 550 train_acc=0.980 test_acc=0.984 train_cost=32.161
Epoch:12 Batch: 550 train_acc=0.980 test_acc=0.983 train_cost=31.916
Epoch:13 Batch: 550 train_acc=1.000 test_acc=0.985 train_cost=25.750
Epoch:14 Batch: 550 train_acc=0.990 test_acc=0.984 train_cost=19.147
Epoch:15 Batch: 550 train_a